In [17]:
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Optional
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import LabelEncoder
import streamlit as st
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
import logging

In [18]:
df_2020 = pd.read_csv('../app/data/pede_2020.csv', sep=';')
df_2021 = pd.read_csv('../app/data/pede_2021.csv', sep=';')
df_2022 = pd.read_csv('../app/data/pede_2022.csv', sep=';')
df_tot = pd.read_csv('../app/data/pede_total.csv', sep=';')

In [19]:
print(df_2020['PONTO_VIRADA'].unique())
print(df_2021['PONTO_VIRADA'].unique())
print(df_2022['PONTO_VIRADA'].unique())

select_2020 = df_2020['PONTO_VIRADA'] == 'Sim' 
select_2021 = df_2021['PONTO_VIRADA'] == 'Sim' 
select_2022 = df_2022['PONTO_VIRADA'] == 'Sim' 

select_2020 = df_2020['PONTO_VIRADA'] == 'Não' 
select_2021 = df_2021['PONTO_VIRADA'] == 'Não' 
select_2022 = df_2022['PONTO_VIRADA'] == 'Não' 
select_sim = lambda x: x== 'Sim'
select_nao = lambda x: x == 'Não'

print(df_2020['PONTO_VIRADA'].apply(select_sim).sum())
print(df_2021['PONTO_VIRADA'].apply(select_sim).sum())
print(df_2022['PONTO_VIRADA'].apply(select_sim).sum())

quantidade_alunos = [ df_2020['PONTO_VIRADA'].apply(select_sim).sum(), df_2020['PONTO_VIRADA'].apply(select_nao).sum(),
                     df_2021['PONTO_VIRADA'].apply(select_sim).sum(), df_2021['PONTO_VIRADA'].apply(select_nao).sum(),
                     df_2022['PONTO_VIRADA'].apply(select_sim).sum(), df_2022['PONTO_VIRADA'].apply(select_nao).sum()]

data = {
    'Ano': [2020, 2020, 2021, 2021, 2022, 2022],
    'Categoria': ['Sim', 'Não', 'Sim', 'Não', 'Sim', 'Não'],
    'Quantidade de Alunos': quantidade_alunos
}

data


['Não' 'Sim' nan]
['Não' 'Sim']
['Não' 'Sim']
94
108
113


{'Ano': [2020, 2020, 2021, 2021, 2022, 2022],
 'Categoria': ['Sim', 'Não', 'Sim', 'Não', 'Sim', 'Não'],
 'Quantidade de Alunos': [np.int64(94),
  np.int64(607),
  np.int64(108),
  np.int64(576),
  np.int64(113),
  np.int64(749)]}

In [20]:
print(df_2020['INSTITUICAO_ENSINO_ALUNO'].unique())
print(df_2021['INSTITUICAO_ENSINO_ALUNO'].unique())
print(df_2022['BOLSISTA'].unique())


bolsista_publica_2020 = df_2020['INSTITUICAO_ENSINO_ALUNO'].apply(lambda x: x == 'Escola Pública' ).sum()
bolsista_publica_2021 = df_2021['INSTITUICAO_ENSINO_ALUNO'].apply(lambda x: x == 'Escola Pública' ).sum()
bolsista_publica_2022 = df_2022['BOLSISTA'].apply(lambda x: x != 'Sim' ).sum()

n_bolsista_publica_2020 = df_2020['INSTITUICAO_ENSINO_ALUNO'].apply(lambda x: x != 'Escola Pública' ).sum()
n_bolsista_publica_2021 = df_2021['INSTITUICAO_ENSINO_ALUNO'].apply(lambda x: x != 'Escola Pública' ).sum()
n_bolsista_publica_2022 = df_2022['BOLSISTA'].apply(lambda x: x == 'Sim' ).sum()

data = {
    'Ano': [2020, 2021, 2022],
    'Bolsista': [bolsista_publica_2020, bolsista_publica_2021, bolsista_publica_2022],
    'Escola Pública': [n_bolsista_publica_2020, n_bolsista_publica_2021, n_bolsista_publica_2022]
}

data


['Escola Pública' 'Rede Decisão/União' 'Einstein' 'Estácio' 'UNISA' 'FIAP'
 'Escola João Paulo II']
['Rede Decisão' 'Escola Pública' 'Escola JP II']
['Não' 'Sim']


{'Ano': [2020, 2021, 2022],
 'Bolsista': [np.int64(598), np.int64(555), np.int64(754)],
 'Escola Pública': [np.int64(129), np.int64(129), np.int64(108)]}

In [21]:
df_2022['FASE'].unique()
df_2021['FASE'].unique()
df_2020['FASE'].unique()

def gerar_estrutura_por_fase(dfs, anos):
    data = {}
    for df, ano in zip(dfs, anos):
        # Conta a quantidade de cada fase (0 a 8)
        fases = df['FASE'].value_counts().reindex(range(9), fill_value=0)
        data[str(ano)] = fases.tolist()
    return data

dfs = [df_2020, df_2021, df_2022]
anos = [2020, 2021, 2022]

# Chama a função e imprime o resultado
estrutura = gerar_estrutura_por_fase(dfs, anos)

estrutura


{'2020': [82, 172, 155, 122, 55, 54, 30, 33, 24],
 '2021': [120, 136, 162, 115, 59, 50, 23, 19, 0],
 '2022': [190, 192, 155, 148, 76, 60, 18, 23, 0]}

In [22]:
print(df_2020['PEDRA'].unique())
print(df_2021['PEDRA'].unique())
print(df_2022['PEDRA'].unique())

def gerar_estrutura_pedras(dfs, anos):
    registros = []
    
    for df, ano in zip(dfs, anos):
        # Agrupa e soma a quantidade de cada tipo de pedra
        agrupamento = df.groupby('PEDRA').size().reindex(['Quartzo', 'Ágata', 'Ametista', 'Topázio'], fill_value=0)
        
        # Monta os registros
        for pedra, quantidade in agrupamento.items():
            registros.append({'Ano': ano, 'Categoria': pedra, 'Quantidade de Alunos': quantidade})

    # Cria o DataFrame final
    return pd.DataFrame(registros)


dfs = [df_2020, df_2021, df_2022]
anos = [2020, 2021, 2022]

# Gera a estrutura
dados_df = gerar_estrutura_pedras(dfs, anos)

dados_df

['Ametista' 'Quartzo' 'Topázio' 'Ágata']
['Ametista' 'Topázio' 'Quartzo' 'Ágata']
['Ametista' 'Quartzo' 'Topázio' 'Ágata']


,Ano,Categoria,Quantidade de Alunos
0,2020,Quartzo,128
1,2020,Ágata,171
2,2020,Ametista,336
3,2020,Topázio,92
4,2021,Quartzo,110
5,2021,Ágata,178
6,2021,Ametista,295
7,2021,Topázio,101
8,2022,Quartzo,134
9,2022,Ágata,250


In [23]:
def plot_media_indicadores(dfs, anos):

    # DataFrame consolidado para médias
    dados_consolidados = pd.DataFrame()

    for df, ano in zip(dfs, anos):
        medias = df.mean()
        dados_consolidados[ano] = medias
    
    return dados_consolidados
    # # Configura o gráfico
    # fig, ax = plt.subplots(figsize=(12, 6))

    # # Plotando cada indicador
    # for indicador in dados_consolidados.index:
    #     ax.plot(anos, dados_consolidados.loc[indicador], marker='o', label=indicador)

    # # Estilização do gráfico
    # ax.set_title('Média dos Indicadores por Ano', fontsize=14)
    # ax.set_xlabel('Ano', fontsize=12)
    # ax.set_ylabel('Média dos Indicadores', fontsize=12)
    # ax.legend(title="Indicadores", bbox_to_anchor=(1.05, 1), loc='upper left')
    # ax.grid(True, linestyle='--', linewidth=0.5)

    # plt.tight_layout()
    # plt.show()


colunas_indicadores = ['INDE', 'IAA', 'IEG', 'IPS', 'IDA', 'IPP', 'IPV', 'IAN']

df_con = plot_media_indicadores([df_2020[colunas_indicadores], df_2021[colunas_indicadores], df_2022[colunas_indicadores]], [2020, 2021, 2022])

df_con


,2020,2021,2022
INDE,7.296111,6.888016,7.028338
IAA,8.368882,8.155702,8.263065
IEG,7.681155,6.842544,7.880728
IPS,6.737448,6.848977,6.901102
IDA,6.322260,5.430556,6.070688
IPP,7.067859,7.598538,6.299364
IPV,7.242243,7.429240,7.247953
IAN,7.431224,6.900585,6.421114
